Step 1: Fill out the form below. 
Step 2: Click the Play button in the top right corner. 
Step 3: Download the export.csv on the left corner

In [ ]:
#@markdown Export for tag comparisons (with examples below)

# Which fields do you want to encode?
# Tag fields -> Which fields do you want to encode
# Text fields -> to compare with
# https://cloud.relevance.ai/sdk/api
authorizationToken="" #@param {type: "string"}
dataset_id="whereto-survey-poc"  #@ param{ type: "string"}
# @markdown Tag fields are split up and converted into 1s and 0s
tag_fields=["B9_taxonomy", "B10_taxonomy"] # @param
number_of_top_tags=10 # @param {type: "integer"}
# @markdown Text fields and cluster fields rae returned with aliases
text_fields=["B9", "B10", "ID. ID", "Serial. Serial number"] # @param
# @markdown Cluster fields are re-mapped with the correct cluster labels
cluster_fields = "['_cluster_.B10_all-mpnet-base-v2_vector_.kmeans-30',  '_cluster_.B9_all-mpnet-base-v2_vector_.kmeans-30', ]" # @param {type: "string"}

cluster_fields = eval(cluster_fields)
# config = dict(
#   authorizationToken="379a70fa08457e03a036:eXBUNXU0QUJQMWlhSktfdy1ScVo6V2I3a0k4UmdTaXUxLURwRHREVVUtUQ:ap-southeast-2:UYqAeDnuUqUEOf1rYrFC5zyjLLL2", #@param {type: "string"}
#   dataset_id="whereto-survey-poc",  #@ param{ type: "string"}
#   tag_fields=["B9_taxonomy", "B4_taxonomy", "D1_taxonomy", "D10_taxonomy", "D4_taxonomy"], # @param
#   text_fields=["B9", "B4", "D1", "D10", "D4"],
#   cluster_fields=["_cluster_.B10_all-mpnet-base-v2_vector_.kmeans-30"],
#   number_of_top_tags=10
# )
!pip install -U -q RelevanceAI-dev
from relevanceai import Client
client = Client(authorizationToken)
ds = client.Dataset(dataset_id)

details = ds.datasets.details(
    dataset_id,
    include_settings=True,
    include_stats=False
)

try:
  field_aliases = details['settings']['fieldAliases']
except:
  field_aliases = {}

import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer

df = ds.to_dict(
    select_fields=tag_fields+text_fields+cluster_fields
)
df = pd.json_normalize(df)
# One hot encode the tag fields
for c in tag_fields:
  results = ds.aggregate(groupby=[{"name": c, "field": c, "agg": "category"}])
  top_tags = [r[c] for r in results['results']]  
  vect = CountVectorizer()
  vect.fit(top_tags)
  values = vect.transform([" ".join(x) if isinstance(x, list) else "" for x in df[c].tolist()]).toarray()
  for i, t in enumerate(top_tags):
    df[field_aliases.get(c, c) + f" ({t})"] = values[:, i]
  df = df.drop(c, axis=1)

for c in cluster_fields:
  cluster_labels = metadata['cluster_metadata']['labels'][c]['labels']
  cluster_labels = {k: v['text'] for k, v in cluster_labels.items()}
  df[c] = df[c].map(cluster_labels)
  df[field_aliases.get(c, c)] = df[c]
  df.drop(c, axis=1, inplace=True)

df.to_csv("export.csv")